# MHW Event 3D

In [1]:
# imports
import numpy as np
import os
from matplotlib import pyplot as plt

from datetime import date

import pandas
import sqlalchemy
from mayavi import mlab

# Load

In [2]:
mhw_file = '/home/xavier/Projects/Oceanography/MHW/db/mhws_allsky_defaults.db'
tst_file = '/home/xavier/Projects/Oceanography/MHW/db/test_mhws_allsky.db'
#
engine = sqlalchemy.create_engine('sqlite:///'+mhw_file)
connection = engine.connect()
connection

In [3]:
mhw_events = pandas.read_sql_table('MHW_Events', con=engine, 
                               columns=['date', 'lon', 'lat', 'duration' , 
                                        'ievent', 'time_peak', 'time_start'])
mhw_events.head()

,date,lon,lat,duration,ievent,time_peak,time_start
0,1982-01-01,0.125,-71.375,13,0,723552,723546
1,1982-12-20,0.125,-71.375,33,1,723901,723899
2,1985-01-30,0.125,-71.375,10,2,724677,724671
3,1988-01-10,0.125,-71.375,9,3,725752,725746
4,1991-01-09,0.125,-71.375,52,4,726858,726841


In [4]:
mhw_events = mhw_events.set_index('date')
mhw_events.head()

,lon,lat,duration,ievent,time_peak,time_start
date,,,,,,
1982-01-01,0.125,-71.375,13,0,723552,723546
1982-12-20,0.125,-71.375,33,1,723901,723899
1985-01-30,0.125,-71.375,10,2,724677,724671
1988-01-10,0.125,-71.375,9,3,725752,725746
1991-01-09,0.125,-71.375,52,4,726858,726841


## Add start date

In [5]:
mhw_events['peak_date'] = pandas.to_datetime([date.fromordinal(tt) for tt in mhw_events['time_peak']])

In [6]:
mhw_events.head()

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
1982-01-01,0.125,-71.375,13,0,723552,723546,1982-01-07
1982-12-20,0.125,-71.375,33,1,723901,723899,1982-12-22
1985-01-30,0.125,-71.375,10,2,724677,724671,1985-02-05
1988-01-10,0.125,-71.375,9,3,725752,725746,1988-01-16
1991-01-09,0.125,-71.375,52,4,726858,726841,1991-01-26


# Pick one

## Not so far from the equator

In [7]:
equ_events = mhw_events[np.abs(mhw_events['lat']) < 20.]

In [8]:
equ_events.iloc[np.argsort(equ_events['duration'])[-20:]]

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2015-03-21,190.375,-2.125,433,55,736005,735678,2016-02-11
2015-03-20,190.125,-2.375,434,51,735997,735677,2016-02-03
2015-03-20,190.125,-2.625,434,51,736015,735677,2016-02-21
1997-06-27,270.625,-11.875,436,9,729351,729202,1997-11-23
1997-08-20,262.375,-10.875,436,10,729459,729256,1998-03-11
1997-08-19,262.625,-10.625,436,12,729417,729255,1998-01-28
1997-06-27,270.625,-12.125,436,11,729350,729202,1997-11-22
2015-03-13,219.875,4.875,438,69,735727,735670,2015-05-09
1997-06-25,270.625,-11.625,438,13,729351,729200,1997-11-23


In [9]:
ievent = np.argsort(equ_events['duration'])[-20:][0]
ievent

8385907

## Build a Cube

### Cut out near my event

In [10]:
event = equ_events.iloc[ievent:ievent+1]
event

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2015-03-21,190.375,-2.125,433,55,736005,735678,2016-02-11


In [11]:
start_date = event.index.values[0]
start_date

numpy.datetime64('2015-03-21T00:00:00.000000000')

In [12]:
end_date= start_date + pandas.Timedelta(days=event.duration.values[0])
end_date

Timestamp('2016-05-27 00:00:00')

## Just use start for now

### Time

In [13]:
#neartime_events = mhw_events["2018-10-15":"2019-02-23"]
neartime_events = equ_events["2015-03-21":"2016-05-27"]

In [14]:
neartime_events.head()

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2015-12-08,0.125,-19.875,9,62,735944,735940,2015-12-12
2016-01-26,0.125,-19.875,7,63,735994,735989,2016-01-31
2016-02-05,0.125,-19.875,47,64,736018,735999,2016-02-24
2015-12-08,0.125,-19.625,9,64,735944,735940,2015-12-12
2016-01-26,0.125,-19.625,7,65,735994,735989,2016-01-31


### Space

In [15]:
event.lat.values[0]

-2.125

In [16]:
nearst_events = neartime_events[(np.abs(
    neartime_events.lat-event.lat.values[0]) < 10.) & (np.abs(
    neartime_events.lon-event.lon.values[0]) < 10.)]
nearst_events.head()

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2015-03-31,180.625,-11.875,7,72,735689,735688,2015-04-01
2015-08-09,180.625,-11.875,5,73,735820,735819,2015-08-10
2015-11-05,180.625,-11.875,9,74,735909,735907,2015-11-07
2015-12-25,180.625,-11.875,5,75,735959,735957,2015-12-27
2016-01-26,180.625,-11.875,14,76,735991,735989,2016-01-28


### Cube

In [17]:
deg = 10.
step = 0.25
grid_lon = np.arange(event.lon.values[0]-deg, event.lon.values[0]+deg+step, step)
grid_lat = np.arange(event.lat.values[0]-deg, event.lat.values[0]+deg+step, step)

In [18]:
cube = np.zeros((event.duration.values[0], grid_lat.size, grid_lon.size), dtype='int')
cube.shape

(433, 81, 81)

In [19]:
for index, row in nearst_events.iterrows():
    #import pdb; pdb.set_trace()
    ilat = np.argmin(np.abs(grid_lat-row.lat))
    jlon = np.argmin(np.abs(grid_lon-row.lon))
    # Do it!
    t0 = row.time_start-event.time_start.values[0]
    cube[t0:t0+row.duration+1, ilat, jlon] = 1

In [20]:
np.sum(cube)/cube.size

0.511879455653869

In [21]:
cube.size

2840913

## Viz

In [22]:
x, y, z = np.ogrid[-10:10:20j, -10:10:20j, -10:10:20j]
s = np.sin(x*y*z)/(x*y*z)

In [23]:
#mlab.contour3d(s)

### Save

In [24]:
np.save('tst_cube.npy', cube)